this script builds research profile of countries after internationally funded publications from a **specific** country is removed. Here the removing standard is remove a paper that funding from the specific country flows to other countries, e.g., USA funded paper is authored by China. 

**all countries are removed iteratively**

In [1]:
import pandas as pd
import tqdm

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pub.pkl"
cntry_full_path="../../data/nf_folder/Data/DerivedData/Derived/cntry_author_full.csv"

profile_path="../../../data/nf_folder/Data/DerivedData/Derived/dep-FundExchange/pub_no_foreign_individual_allcountries.csv"

In [3]:
# Parameters
pubs_path = "../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"
cntry_full_path = (
    "../data/NationalFunding/Data/DerivedData/Derived/cntry_author_full.csv"
)
profile_path = "../data/NationalFunding/Data/DerivedData/Derived/dependence/pub_noforeign_fund_all2all.csv"


In [4]:
pub_df=pd.read_pickle(pubs_path)
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[Canada, United States]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[France, Chile]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


In [5]:
def get_publications_to_remove(fund_df,cntry):
    """Returns a list of publication IDs that are funded by a specific country
    and are either internationally coauthored or authored by other countries."""
    def assign_label(authors, cntry):
        
        if len(authors)>1 or authors[0]!=cntry:
            return 1
        else:
            return 0
    
    df=fund_df[fund_df.funder_distinct==cntry].copy()
    df['label']=df.apply(lambda x: assign_label(x.author_distinct,cntry),axis=1)
    removeid=list(df[df.label==1]['id'].unique())
    
    return removeid

In [6]:
def build_country_profile(pub_df, fund_df, cntry):
    """construct the research profile of rest of countries when the internationally-funded publications are removed
    """
    
    removeid=get_publications_to_remove(fund_df, cntry)
    df=pub_df[~pub_df['id'].isin(removeid)]
    df=df[['id','dis','author_distinct']].explode('author_distinct') #assign papers to authorship countries using full counting
    df=df.groupby(['author_distinct','dis']).size().reset_index(name='count')
    
    return df

In [7]:
filter_df=pd.DataFrame()
fund_df = pub_df[['id','author_distinct','funder_distinct']].explode('funder_distinct')
cntrys=fund_df.funder_distinct.unique()

for cntry in tqdm.tqdm(cntrys):
    country_profile=build_country_profile(pub_df, fund_df, cntry)
    country_profile['srce_cntry']=cntry
    filter_df=pd.concat([filter_df,country_profile])

  0%|                                                   | 0/192 [00:00<?, ?it/s]

  1%|▏                                        | 1/192 [00:26<1:24:03, 26.40s/it]

  1%|▍                                        | 2/192 [01:18<2:10:45, 41.29s/it]

  2%|▋                                        | 3/192 [01:27<1:24:29, 26.82s/it]

  2%|▊                                        | 4/192 [01:38<1:03:43, 20.34s/it]

...

 98%|████████████████████████████████████████▎| 189/192 [31:05<00:27,  9.32s/it]

 99%|████████████████████████████████████████▌| 190/192 [31:14<00:18,  9.26s/it]

 99%|████████████████████████████████████████▊| 191/192 [31:23<00:09,  9.22s/it]

100%|█████████████████████████████████████████| 192/192 [31:32<00:00,  9.22s/it]

100%|█████████████████████████████████████████| 192/192 [31:32<00:00,  9.86s/it]

In [8]:
filter_df=filter_df.groupby(['author_distinct','srce_cntry'])['count'].sum().reset_index()
filter_df=filter_df.rename(columns={'count':'filter'})

In [9]:
raw_df=pd.read_csv(cntry_full_path)
raw_df = raw_df.groupby(['cntry'])['count'].sum().reset_index(name='raw')

In [10]:
meta=raw_df.merge(filter_df,left_on='cntry', right_on='author_distinct')
meta['p']=(meta['raw']-meta['filter'])/meta['raw']

In [11]:
meta=meta[['srce_cntry','cntry','raw','filter','p']]
meta=meta.rename(columns={'srce_cntry':'source','cntry':'target','raw':'raw_pub','filter':'filter_pub'})
meta=meta[meta.source!='Not-Funded']

In [12]:
meta.to_csv(profile_path, index=False)